In [1]:
# Import dependencies

import numpy as np
import pandas as pd
import requests
import json
import time

from scipy.stats import linregress
from matplotlib import pyplot as plt

from citipy import citipy

In [2]:
# Import OpenWeather API key

from config import api_key

In [3]:
# Generate a random list of cities with citipy

# Generate a random list of coordinates
# "The valid range of latitude in degrees is -90 and +90 for the southern and northern hemisphere respectively. Longitude is in the range -180 and +180" https://stackoverflow.com/questions/15965166/

latitude = []
longitude = []

for lat in np.random.randint(-90,90,500):
    latitude.append(lat)

for long in np.random.randint(-180,180,500):
    longitude.append(long)

lat_long = []
for i in range(len(latitude)):
    lat_long.append((latitude[i], longitude[i]))

# lat_long = tuple(zip(latitude,longitude))

# lat_long

# Locate nearest cities to coordinates
cities = []

for lat,long in lat_long:
    city = citipy.nearest_city(lat,long)
    city_name = city.city_name
    cities.append(city_name)

# cities

In [ ]:
# Testing data retrieval

base_url = "http://api.openweathermap.org/data/2.5/weather?".format(target_city,api_key)
target_url = base_url + "q=" + city_name + "&appid=" + api_key

city_name = "albany"

print("Tesing OpenWeatherMap API")

response = requests.get(target_url)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

data_test = response.json()
# print(json.dumps(data_test, indent=4, sort_keys=True))

main = data_test["main"]
temperature = main["temp"] # Fetch temperature
humidity = main["humidity"] # Fetch humidity

wind = data_test["wind"] 
wind_speed = wind["speed"] # Fetch wind speed

clouds = data_test["clouds"]
cloudiness = clouds["all"] # Fetch cloudiness


print(f"{city_name.title():-^30}")
print(f"Temperature: {temperature}")
print(f"Humidity: {humidity}")
print(f"Wind speed: {wind_speed}")
print(f"Cloudiness: {cloudiness}")


In [4]:
# Testing data retrieval method 2 (using parameter dictionary)

base_url = "http://api.openweathermap.org/data/2.5/weather"
params = {
    "q": "albany",
    "appid": api_key
}
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

data_test = response.json()

temperature = data_test["main"]["temp"] # Fetch temperature
humidity = data_test["main"]["humidity"] # Fetch humidity

# wind = data_test["wind"] 
# wind_speed = wind["speed"] # Fetch wind speed
wind_speed = data_test["wind"]["speed"] # Fetch wind speed


# clouds = data_test["clouds"]
# cloudiness = clouds["all"] # Fetch cloudiness
cloudiness = data_test["clouds"]["all"] # Fetch cloudiness


print(f"{city_name.title():-^30}")
print(f"Temperature: {temperature}")
print(f"Humidity: {humidity}")
print(f"Wind speed: {wind_speed}")
print(f"Cloudiness: {cloudiness}")


------------Albany------------
Temperature: 274.56
Humidity: 64
Wind speed: 3.58
Cloudiness: 97


In [15]:
# Testing data retrieval method 3 (adding parameter for metric units of measurement)

base_url = "http://api.openweathermap.org/data/2.5/weather"
params = {
    "q": "albany",
    "appid": api_key,
    "units": "metric"
}
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

data_test = response.json()

temperature = data_test["main"]["temp"] # Fetch temperature
humidity = data_test["main"]["humidity"] # Fetch humidity

# wind = data_test["wind"] 
# wind_speed = wind["speed"] # Fetch wind speed
wind_speed = data_test["wind"]["speed"] # Fetch wind speed


# clouds = data_test["clouds"]
# cloudiness = clouds["all"] # Fetch cloudiness
cloudiness = data_test["clouds"]["all"] # Fetch cloudiness


print(f"{city_name.title():-^30}")
print(f"Temperature: {temperature}")
print(f"Humidity: {humidity}")
print(f"Wind speed: {wind_speed}")
print(f"Cloudiness: {cloudiness}")


------------Albany------------
Temperature: 1.37
Humidity: 63
Wind speed: 2.68
Cloudiness: 97


In [5]:
# Set up a DataFrame to hold city, lat, long, temperature, humidity, wind speed and cloudiness 
weather_df = pd.DataFrame(cities, columns=["city"])
weather_df.head()
# weather_df.info()


# Set up additional columns to hold information
weather_df['latitude'] = latitude
weather_df['longitude'] = longitude
weather_df['temperature'] = ""
weather_df['humidity'] = ""
weather_df['wind speed'] = ""
weather_df['cloudiness'] = ""

weather_df

,city,latitude,longitude,temperature,humidity,wind speed,cloudiness
0,longyearbyen,86,16,,,,
1,butaritari,23,168,,,,
2,cape town,-56,3,,,,
3,ushuaia,-57,-64,,,,
4,isangel,-25,177,,,,
...,...,...,...,...,...,...,...
495,vostok,48,150,,,,
496,saskylakh,77,114,,,,
497,torbay,43,-45,,,,
498,amderma,71,64,,,,


In [19]:
# Data retrieval

base_url = "http://api.openweathermap.org/data/2.5/weather"
params = {
    "q": "albany",
    "appid": api_key,
    "units": "metric"
}

# Use iterrows to iterate through pandas dataframe
for index, row in weather_df.iterrows():
# for index, row in weather_df.head(n=5).iterrows(): # brief test

    # Get city from weather_df
    city = row['city']

    # Add query to params dict
    params['q'] = city

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}")

    # Extract results
    response = requests.get(base_url, params=params).json()
    
    try:
        weather_df.loc[index, "temperature"] = response["main"]["temp"]
        weather_df.loc[index, "humidity"] = response["main"]["humidity"] 
        weather_df.loc[index, "wind speed"] = response["wind"]["speed"]
        weather_df.loc[index, "cloudiness"] = response["clouds"]["all"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


g Results for Index 161: wahiawa
------------
Retrieving Results for Index 162: jamestown
------------
Retrieving Results for Index 163: buraydah
------------
Retrieving Results for Index 164: khuzdar
------------
Retrieving Results for Index 165: ushuaia
------------
Retrieving Results for Index 166: jamestown
------------
Retrieving Results for Index 167: jamestown
------------
Retrieving Results for Index 168: victoria
------------
Retrieving Results for Index 169: oranjemund
------------
Retrieving Results for Index 170: airai
------------
Retrieving Results for Index 171: panguna
------------
Retrieving Results for Index 172: atuona
------------
Retrieving Results for Index 173: carnarvon
------------
Retrieving Results for Index 174: tuktoyaktuk
------------
Retrieving Results for Index 175: yulara
------------
Retrieving Results for Index 176: ambilobe
------------
Retrieving Results for Index 177: akhisar
------------
Retrieving Results for Index 178: albany
------------
Retrie

In [20]:
# Check data in DataFrame
weather_df

,city,latitude,longitude,temperature,humidity,wind speed,cloudiness
0,longyearbyen,86,16,1,69,17.5,75
1,butaritari,23,168,27.33,79,9.26,100
2,cape town,-56,3,25.99,57,5.7,0
3,ushuaia,-57,-64,12,71,2.1,75
4,isangel,-25,177,25.06,84,3.85,98
...,...,...,...,...,...,...,...
495,vostok,48,150,-4.52,99,1.18,62
496,saskylakh,77,114,-12.46,94,4.68,98
497,torbay,43,-45,10,87,14.4,90
498,amderma,71,64,,,,


In [21]:
# Save data to CSV
weather_df.to_csv("weather.csv")